In [ ]:
import os
import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point
import time

In [ ]:
zones_gdf = gpd.read_file(zones_path)

In [ ]:
# Параметры
token = ""
keywords = ["парк", "сад", "лесопарк", "сквер"]
total_required_posts_per_keyword = 1500  # Нужное количество постов для каждого ключевого слова
count_per_request = 200  # Количество постов за один запрос
end_time = int(time.time())
latitude = 59.94
longitude = 30.31

all_posts = []

In [ ]:
for keyword in keywords:
    print(f"Сбор данных для ключевого слова: {keyword}")
    collected_posts = []
    current_end_time = end_time

    while len(collected_posts) < total_required_posts_per_keyword:
        # Формируем запрос
        url = f"https://api.vk.com/method/newsfeed.search?q={keyword}&count={count_per_request}&latitude={latitude}&longitude={longitude}&end_time={current_end_time}&access_token={token}&v=5.131"
        print(f"Выполняется запрос: {url}")

        # Отправка запроса
        response = requests.get(url)
        data = response.json()

        # Проверка ответа
        if "response" in data:
            posts = data.get('response', {}).get('items', [])

            # Если постов нет, завершаем цикл для этого ключевого слова
            if not posts:
                print("Больше постов нет в указанном интервале.")
                break

            # Фильтрация: оставляем только посты с полем 'geo', которое не равно None
            filtered_posts = [post for post in posts if 'geo' in post and post['geo'] is not None]

            # Преобразуем geo в координаты и проверяем включение в полигоны зелёных зон
            valid_posts = []
            for post in filtered_posts:
                try:
                    # Преобразуем координаты из geo
                    lat, lon = map(float, post['geo']['coordinates'].split())
                    point = Point(lon, lat)

                    # Проверяем, находится ли точка внутри любой зоны
                    if zones_gdf.contains(point).any():
                        valid_posts.append(post)

                except Exception as e:
                    print(f"Ошибка обработки координат: {e}")

            # Добавляем отфильтрованные посты к списку
            collected_posts.extend(valid_posts)

            # Обновляем end_time на время последнего поста, минус одна секунда
            current_end_time = posts[-1]["date"] - 1

            print(f"Собрано постов для ключевого слова '{keyword}': {len(collected_posts)}")

        else:
            print("Ошибка в запросе:", data.get("error"))
            break

        # Задержка, чтобы избежать блокировки
        time.sleep(1)

    # Добавляем собранные посты для текущего ключевого слова в общий список
    all_posts.extend(collected_posts)

In [ ]:
# Преобразование в GeoDataFrame
df = pd.DataFrame(all_posts)
df['latitude'] = df['geo'].apply(lambda x: float(x['coordinates'].split()[0]) if x and 'coordinates' in x else None)
df['longitude'] = df['geo'].apply(lambda x: float(x['coordinates'].split()[1]) if x and 'coordinates' in x else None)
df = df.dropna(subset=['latitude', 'longitude'])
df['geometry'] = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(f"Всего собрано постов: {len(gdf)}")
gdf